### Loading Dataset

In [1]:
!pip install opendatasets
import opendatasets as od

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
username:"zahrasorkhei"
key:"0548e34bf61c0f9a1661511e47b029e6"
od.download(
    'https://www.kaggle.com/datasets/odedgolden/movielens-1m-dataset/u.data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: zahrasorkhei
Your Kaggle Key: ··········


100%|██████████| 5.83M/5.83M [00:00<00:00, 126MB/s]

In [3]:
import pandas as pd
# Make display smaller
pd.options.display.max_rows = 10
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('/content/movielens-1m-dataset/users.dat', sep='::',
 header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('/content/movielens-1m-dataset/ratings.dat', sep='::',
 header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('/content/movielens-1m-dataset/movies.dat', sep='::',
 header=None, names=mnames, engine='python' , encoding="ISO-8859-1")

### Question 17

In [4]:
import numpy as np
d = ratings['user_id'].value_counts()
for i in d:
  if(d[i]<20):
    ratings.drop(ratings.index[ratings['user_id'] == i ], inplace = True)

### Question 18, 19 and 20

In [5]:
### Find X for some users
def findX(ratings , users , movies , len_user_total , len_movie_total):
  X = np.zeros((len_user_total, len_movie_total ))
  for index, row in users.iterrows():
    userID = row['user_id']
    rateing_of_user = ratings.loc[ratings['user_id'] == userID]
    for index2, row2 in rateing_of_user.iterrows():
      X[userID-1 , row2['movie_id']-1] = row2['rating']
  return X


In [6]:
def training(X_train, lambda_ ):
    G = X_train.T.dot(X_train)
    #diagIndices = np.diag_indices(G.shape[0])
    G += np.identity(G.shape[0])*lambda_
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    B = B - np.diag(np.diag(B))
    return B


In [7]:
def testing(X_test, B):
  return X_test.dot(B)

In [8]:
from sklearn.metrics import r2_score
from statistics import mean
def cross_validation(ratings,users,movies):
  best_lambda_ = 0
  score_best = -1000
  lambda_ = [0.5 , 100 , 500 , 1000 , 4000 , 7000 , 8000 , 10000]
  Best_B = 0
  len_user_total = users['user_id'].max()
  len_movie_total = movies['movie_id'].max()
  X = findX(ratings , users , movies , len_user_total , len_movie_total )
  # Spliting to test and train part
  X_train = X[:int(X.shape[0]*0.8),:]
  X_test = X[int(X.shape[0]*0.8):,:]
  for i in range(len(lambda_)):
      print("\n========================Step ",i)
      XX = np.zeros((X_train.shape[0], X_train.shape[1] ))
      # Shuffle the X randomly 
      np.take(X_train,np.random.permutation(X_train.shape[0]),axis=0,out=XX)
      score = [] 
      for j in range(5):
        # Split train_data to train and validation
        idx_test = np.arange(0*XX.shape[0],int((0+1)*XX.shape[0]*0.2) ,1)
        idx_train = list(set(np.arange(0,XX.shape[0] ,1)) - set(idx_test))
        XX_train = XX[idx_train]
        XX_test =XX[idx_test]
        #Train and find Beta
        B = training(XX_train, lambda_[i] )
        #Test 
        X_test_prim = testing(XX_test, B)
        #R2 score
        score.append(r2_score(XX_test.flatten(), X_test_prim.flatten()))
      print(mean(score))
      if(mean(score) > score_best):
        score_best = mean(score)
        Best_B = B
      print("Best score until step ",i," =",score_best)
  X_test_prim = testing(X_test, Best_B)
  score_test = r2_score(X_test.flatten(), X_test_prim.flatten())
  print("\n\nR2 score on test = ",score_test)
  return best_lambda_ 

In [9]:
best_lambda_  = cross_validation(ratings,users,movies)


========================Step  0
-3.8071266844592904
Best score until step  0  = -3.8071266844592904

========================Step  1
0.06446683467641545
Best score until step  1  = 0.06446683467641545

========================Step  2
0.2948144228705033
Best score until step  2  = 0.2948144228705033

========================Step  3
0.36338273067573035
Best score until step  3  = 0.36338273067573035

========================Step  4
0.41700347585911046
Best score until step  4  = 0.41700347585911046

========================Step  5
0.4201962317947662
Best score until step  5  = 0.4201962317947662

========================Step  6
0.41691183665639797
Best score until step  6  = 0.4201962317947662

========================Step  7
0.4314461585481213
Best score until step  7  = 0.4314461585481213


R2 score on test =  0.39223001406583147
